**APPROCHE METHODOLOGIQUE**
---

Problème de classification (28 classes) avec évaluation "sur mesure" (composite)<br><br>. Jeu de donnée de 25.000 personnes, après voir lancé la baseline, j'ai entrainé un **adversarial NN**, qui permettait de gagner un peu d'équité (passage de 80% à 85% sur le TRP gap) mais a entrainé un éffondrement de l'accuracy (de 65% à 35%). après avoir cherché à personaliser la fonction de perte (avec difficulté pour entrainer le modele), je suis repasser à une approche plus progressive, sur la régression logistique

**regression logisitique (baseline) => 3 problemes**<br>
1. ***l'échantillonage*** : Le "train_test_split" ne tient pas compte de S, nous ne savons pas si l'attribut protégé (H/F) est bien réparti entre X_train et X_test, cela n'est pas optimum pour l'apprentissage car certaines classes contiennent peu de points. Il peut aussi etre plus pertinent de fixer les hyperparametre grace à X_train, et d'entrainer le modele sur tout l'échantillon (avec les memes hyperparametres) avant de prédire X_test_true et de soumettre<br>
=> nous réaliserons des train_test_split en tenant compte de X et Y grace à Y56 = Y + 28*S <br>
=> nous ferons des soumissions double (modele appris sur X_train et aussi X) pour le datachallenge, pour améliorer les qualités prédicitive du modèles <br><br>
2. la fonction d'évaluation est une ***moyenne de scores calculé par classes***, chaque classe a donc le meme poids dans le score final. Or, l'apprentissage est fait pour optimser l'accuracy de l'échantillon (chaque élément à le meme poids). Comme la distribution de l'échatillon est très mal équilibrée entre les classes (93 à 8.285 personnes), le modèle ne peut apprendre correctement<br>
=> nous essaierons pour la forme del a data augmentation, mais sans conviction compte tenu de l'embedding sémantique fait avec BERT<br>
=> nous utiliserons une correction de la fonction de pertes pour y inclure le poids de chaque classe (Y et Y56)<br>
=> nous essaierons du contrastive learning pour obtenir un espace de réprésentation adéquat<br><br>
2. ***l'apprentissage n'ast pas aligné avec le score final***: l'apprentissage est fait pour optimiser l'accuracy (la précision), or la fonction d'évaluation est une moyenne entre le F1 score (moyenne harmonique de la précision et du rappel) et une métrique de fairness. Le F1 score pénalise les écarts importants entre la précision et le rappel. Le score<br>
=> nous adapterons le fonctions de pertes et/ou l'architecture des modèles intégrer au mieux notrescore final<br> 

**Nous allons procéder en 2 temps** :

1/ nous allons **échantilloner X par rapport à Y et S** traiter le probleme 1 et 2, en créant un label à 56 classes (2 x 28) pour distinguer chaque label i en fonction de s'il est protégé ou non. Nous augmenterons la taille de l'échantillon (avec 56 labels) pour créer X_XL et Y_XL pour avoir le meme nombre d'items pour chacune des 56 classes et un jeu avec le meme nombre d'instance par classe, permettant d'aligner les scores moyens des classes avec la moyenne du nouveau sample "XL" décuplé (passage de 27.749 lignes à 255.304 lignes ). Nous programmerons la régression logisitique avec pytorch pour accélérer les calculs.

2/ nous **ajusterons ensuite la fonction de pertes**, pour qu'elle tienne compte au mieux de la fonction d'évaluation, qui est composée de 2 éléments : le F1 score et le 1-equal ooprtunity gap<br>
- le 'macro f score' (F1 score) n'est pas dérivable (à valider), nous l'approcherons donc par accuracy et le recall<br>
- true positive gap : 1 - TRP_GAP est calculé comme 
A noter que le F1 score n'est pas dérivable. However, we can approximate the F1 score in a differentiable manner by using the ***Sørensen–Dice coefficient***, which is closely related to the F1 score and is differentiable. The Sørensen–Dice coefficient is defined as \(2 * \frac{precision * recall}{precision + recall}\), which is equivalent to the F1 score formula. For a differentiable approximation, we can use soft versions of precision and recall.
<br><br>

3/ Nous explorerons **différents modèles de classification** K-means, SVM, random forest tree<br><br>

4/ Avant de mener cette approche j'avais essayer 



In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE

from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

from evaluator_ANAELE import *

'''import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC, SparseCategoricalAccuracy
from tensorflow.keras.callbacks import ReduceLROnPlateau

'''

'import tensorflow as tf\nfrom tensorflow.keras.layers import Input, Dense, Dropout\nfrom tensorflow.keras.models import Model\nfrom tensorflow.keras.optimizers import Adam\nfrom tensorflow.keras.metrics import AUC, SparseCategoricalAccuracy\nfrom tensorflow.keras.callbacks import ReduceLROnPlateau\n\n'

dict_keys(['X_train', 'X_test', 'Y', 'S_train', 'S_test'])
(27749, 768) (27749,) (27749,) (11893, 768) (11893,)


In [3]:
#create a label to distiguish 56 labels Y x 2 (man or woman)
# 0 to 27 = non sensitive group | 28 + [0 , 27] = 28 to 55 = sensitive group
Y56 = Y+28*S

In [4]:
'''label_56=pd.DataFrame({'Y':Y,'S':S})
label_56['label_x2']=Y+28*S
label_56['1']=1
label_56['F']=S
label_56['H']=1-S
total_56 = label_56[['Y','1','F','H']].groupby(['Y']).sum()
total_56'''

'''total2 = X[['1','label_x2','F','H']].groupby(['label_x2']).sum()
total2'''

"total2 = X[['1','label_x2','F','H']].groupby(['label_x2']).sum()\ntotal2"

In [24]:
# FUNCTIONS

# to show performance

def evaluate(Y_pred,Y,S,will_print=1):
    '''returns model accuracy, final score, macro fscore ans TPR gap
    input : 2 np arrays of same dimension
    output : array of 4 values
    '''
    accuracy= accuracy_score(Y, Y_pred)  # Y_test are your original test labels
    print(f"Accuracy on transformed test data: {accuracy}")
    eval_scores, confusion_matrices_eval = gap_eval_scores(Y_pred, Y, S, metrics=['TPR'])
    final_score = (eval_scores['macro_fscore']+ (1-eval_scores['TPR_GAP']))/2

    if will_print==1:
        #print results
        print('final score',final_score)
        print('macro_fscore',eval_scores['macro_fscore'])
        print('1-eval_scores[\'TPR_GAP\']',1-eval_scores['TPR_GAP'])
    
    return accuracy, final_score, eval_scores['macro_fscore'],1-eval_scores['TPR_GAP'] , eval_scores , confusion_matrices_eval

# to predict X_test and save to file

def save_X_test_true(X, model,name):
    Y_pred = model.predict(X)
    results=pd.DataFrame(y_pred, columns= ['score'])
    file_name = "Data_Challenge_MDI_341_"+str(name)+".csv"
    results.to_csv(file_name, header = None, index = None)
    
    return Y_pred



**MODEL 0 = BASELINE (28 classes)**
---
split uniquement sur Y (sans tenir compte de S)

In [6]:
# Split the data (final _ to keep split data untouched and be able to reload in file)
X_train, X_test, Y_train, Y_test, S_train, S_test = train_test_split(X, Y, S, test_size=0.3, random_state=42)

In [8]:
# training logistic model
clf_0 = LogisticRegression(random_state=0, max_iter=5000,verbose=1).fit(X_train, Y_train)

# predicting and assessing
Y_pred_0 = clf_0.predict(X_test)
accuracy_0= accuracy_score(Y_test, Y_pred_0)  # Y_test are your original test labels
print(f"Accuracy on transformed test data: {accuracy_0}")
eval_scores_0, confusion_matrices_eval_0 = gap_eval_scores(Y_pred_0, Y_test, S_test, metrics=['TPR'])
final_score_0 = (eval_scores_0['macro_fscore']+ (1-eval_scores_0['TPR_GAP']))/2

#print results
print('final score',final_score_0)
print('macro_fscore',eval_scores_0['macro_fscore'])
print('1-eval_scores[\'TPR_GAP\']',1-eval_scores_0['TPR_GAP'])

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =        21532     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.33220D+00    |proj g|=  1.49704D+00


 This problem is unconstrained.



At iterate   50    f=  7.05446D-01    |proj g|=  1.61916D-02

At iterate  100    f=  5.50259D-01    |proj g|=  2.34089D-02

At iterate  150    f=  4.73919D-01    |proj g|=  6.63617D-03

At iterate  200    f=  4.36572D-01    |proj g|=  3.26170D-03

At iterate  250    f=  4.15622D-01    |proj g|=  2.62995D-03

At iterate  300    f=  4.06518D-01    |proj g|=  8.17065D-03

At iterate  350    f=  4.01723D-01    |proj g|=  1.90510D-03

At iterate  400    f=  3.99333D-01    |proj g|=  2.77170D-03

At iterate  450    f=  3.97783D-01    |proj g|=  9.01702D-04

At iterate  500    f=  3.96904D-01    |proj g|=  4.49510D-04

At iterate  550    f=  3.96496D-01    |proj g|=  5.93727D-04

At iterate  600    f=  3.96233D-01    |proj g|=  5.38018D-04

At iterate  650    f=  3.96080D-01    |proj g|=  3.29728D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates

In [13]:
accuracy, final_score, macro_fscore , equity_score , eval_scores , confusion_matrices_eval =evaluate(Y_pred_0, Y_test,S_test)

Accuracy on transformed test data: 0.7633633633633634
gap_eval_scores
{'accuracy': 0.7633633633633634, 'macro_fscore': 0.6695276930026747, 'micro_fscore': 0.7633633633633634}
distinct_groups size 2
{'TPR': array([0.62337659, 0.62416106, 0.85783131, 0.47500001, 0.57407405,
       0.80952374, 0.88135589, 0.74999938, 0.7999998 , 0.78571423,
       0.25000062, 0.64705881, 0.42424247, 0.70270259, 0.67010306,
       0.2000012 , 0.51428571, 0.54166663, 0.83285301, 0.78620688,
       0.59493668, 0.87262079, 0.60389609, 0.66666651, 0.49137931,
       0.6048387 , 0.44366198, 0.5999996 ]), 'TNR': array([0.99638663, 0.98783287, 0.98312958, 0.99708846, 0.99842732,
       0.99683329, 0.99422366, 0.99933289, 0.99888268, 0.99500794,
       1.        , 0.98202614, 0.99552773, 0.99597135, 0.99637023,
       0.99933333, 0.99463087, 0.9993305 , 0.98436748, 0.97100737,
       0.99096249, 0.92258681, 0.98942772, 0.99977698, 0.98838004,
       0.98590556, 0.98120559, 0.99977778]), 'PPV': array([0.74999992, 0

In [14]:
# training logistic model
clf_0_all = LogisticRegression(random_state=0, max_iter=5000,verbose=1).fit(X, Y)


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =        21532     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.33220D+00    |proj g|=  1.48160D+00


 This problem is unconstrained.



At iterate   50    f=  7.30229D-01    |proj g|=  1.75925D-02

At iterate  100    f=  5.85159D-01    |proj g|=  7.52089D-03

At iterate  150    f=  5.21361D-01    |proj g|=  1.53492D-02

At iterate  200    f=  4.82218D-01    |proj g|=  6.42231D-03

At iterate  250    f=  4.60058D-01    |proj g|=  1.24479D-02

At iterate  300    f=  4.49281D-01    |proj g|=  4.70459D-03

At iterate  350    f=  4.42280D-01    |proj g|=  5.22195D-03

At iterate  400    f=  4.38594D-01    |proj g|=  2.02481D-03

At iterate  450    f=  4.36522D-01    |proj g|=  3.92893D-03

At iterate  500    f=  4.35307D-01    |proj g|=  7.10307D-04

At iterate  550    f=  4.34608D-01    |proj g|=  2.45725D-03

At iterate  600    f=  4.34215D-01    |proj g|=  4.29856D-04

At iterate  650    f=  4.33949D-01    |proj g|=  3.35770D-04

At iterate  700    f=  4.33793D-01    |proj g|=  8.50265D-04

At iterate  750    f=  4.33698D-01    |proj g|=  2.10926D-04

At iterate  800    f=  4.33630D-01    |proj g|=  2.02935D-04

At iter

In [18]:
Y_pred_0_all = clf_0_all.predict(X)
accuracy, final_score, macro_fscore , equity_score , eval_scores , confusion_matrices_eval =evaluate(Y_pred_0_all, Y,S)

Accuracy on transformed test data: 0.8894014198709863
gap_eval_scores
{'accuracy': 0.8894014198709863, 'macro_fscore': 0.9006861250232686, 'micro_fscore': 0.8894014198709863}
distinct_groups size 2
{'TPR': array([0.86075946, 0.83703702, 0.93994211, 0.79687495, 0.95624994,
       0.98148145, 0.95858894, 0.81818153, 0.98913033, 0.94524493,
       0.99999929, 0.86052998, 0.80645155, 0.74015744, 0.9335664 ,
       0.5714285 , 0.87121206, 0.9636362 , 0.94878705, 0.87140902,
       0.89534881, 0.92520289, 0.74089068, 0.99999988, 0.82097185,
       0.76521738, 0.64399092, 0.95238052]), 'TNR': array([0.99891297, 0.99542175, 0.99160159, 0.99946048, 0.99972966,
       0.99911154, 0.99664429, 1.        , 1.        , 0.99863098,
       1.        , 0.99044877, 0.99973087, 0.99925821, 0.99809134,
       1.        , 0.99912304, 0.99986578, 0.99422091, 0.98265896,
       0.99727854, 0.95777628, 0.99398423, 1.        , 0.99526262,
       0.99208536, 0.99269721, 1.        ]), 'PPV': array([0.92727269, 0

In [19]:
# (Re)Load the "true" test data
X_test_true = dat['X_test']
S_test_true = dat['S_test'] 

# Classify the provided test data with you classifier
y_test_true_0_all = clf_0_all.predict(X_test_true)
results_0_all=pd.DataFrame(y_test_true_0_all, columns= ['score'])

results_0_all.to_csv("Data_Challenge_MDI_341_0_all.csv", header = None, index = None)
# np.savetxt('y_test_challenge_student.txt', y_test, delimiter=',')

In [21]:
# Split the data (final _ to keep split data untouched and be able to reload in file)
X_train, X_test, Y_train, Y_test, S_train, S_test = train_test_split(X, Y, S, test_size=0.3, random_state=1)

# training logistic model
clf_0_1 = LogisticRegression(random_state=0, max_iter=5000,verbose=1).fit(X_train, Y_train)

# predicting and assessing
Y_pred_0_1 = clf_0.predict(X_test)
accuracy_0_1= accuracy_score(Y_test, Y_pred_0_1)  # Y_test are your original test labels
print(f"Accuracy on transformed test data: {accuracy_0_1}")
eval_scores_0_1, confusion_matrices_eval_0_1 = gap_eval_scores(Y_pred_0_1, Y_test, S_test, metrics=['TPR'])
final_score_0_1 = (eval_scores_0_1['macro_fscore']+ (1-eval_scores_0_1['TPR_GAP']))/2

#print results
print('final score',final_score_0_1)
print('macro_fscore',eval_scores_0_1['macro_fscore'])
print('1-eval_scores[\'TPR_GAP\']',1-eval_scores_0_1['TPR_GAP'])

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =        21532     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.33220D+00    |proj g|=  1.49013D+00


 This problem is unconstrained.



At iterate   50    f=  7.00321D-01    |proj g|=  2.45646D-02

At iterate  100    f=  5.42959D-01    |proj g|=  1.33898D-02

At iterate  150    f=  4.64078D-01    |proj g|=  4.90243D-03

At iterate  200    f=  4.28360D-01    |proj g|=  9.46923D-03

At iterate  250    f=  4.08372D-01    |proj g|=  3.70542D-03

At iterate  300    f=  3.99541D-01    |proj g|=  9.83565D-03

At iterate  350    f=  3.94082D-01    |proj g|=  4.78952D-03

At iterate  400    f=  3.90846D-01    |proj g|=  2.24606D-03

At iterate  450    f=  3.89635D-01    |proj g|=  5.83798D-04

At iterate  500    f=  3.88954D-01    |proj g|=  9.84457D-04

At iterate  550    f=  3.88570D-01    |proj g|=  1.04105D-03

At iterate  600    f=  3.88323D-01    |proj g|=  1.25465D-03

At iterate  650    f=  3.88213D-01    |proj g|=  4.94155D-04

At iterate  700    f=  3.88151D-01    |proj g|=  2.91062D-04

At iterate  750    f=  3.88114D-01    |proj g|=  1.85265D-04

           * * *

Tit   = total number of iterations
Tnf   = total nu

In [23]:
# (Re)Load the "true" test data
X_test_true = dat['X_test']
S_test_true = dat['S_test'] 

# Classify the provided test data with you classifier
y_test_true_0_1 = clf_0_1.predict(X_test_true)
results_0_1=pd.DataFrame(y_test_true_0_1, columns= ['score'])

results_0_1.to_csv("Data_Challenge_MDI_341_0_1.csv", header = None, index = None)
# np.savetxt('y_test_challenge_student.txt', y_test, delimiter=',')

**MODEL 1 - BASELINE + SPLIT Y x S**
---

In [30]:
# Split the data (final _ to keep split data untouched and be able to reload in file)
X_train, X_test, Y56_train, Y56_test, S_train, S_test = train_test_split(X, Y56, S, test_size=0.3, random_state=42)
Y_train = Y56_train % 28  # reste (original Y)
Y_test = Y56_test % 28    # reste (original Y)

In [31]:
# training logistic model
clf_1 = LogisticRegression(random_state=0, max_iter=5000,verbose=1).fit(X_train, Y_train)

# predicting and assessing
Y_pred_1 = clf_1.predict(X_test)
accuracy_1= accuracy_score(Y_test, Y_pred_1)  # Y_test are your original test labels
print(f"Accuracy on transformed test data: {accuracy_1}")
eval_scores_1, confusion_matrices_eval_1 = gap_eval_scores(Y_pred_1, Y_test, S_test, metrics=['TPR'])
final_score_1 = (eval_scores_1['macro_fscore']+ (1-eval_scores_1['TPR_GAP']))/2

#print results
print('final score',final_score_1)
print('macro_fscore',eval_scores_1['macro_fscore'])
print('1-eval_scores_1[\'TPR_GAP\']',1-eval_scores_1['TPR_GAP'])

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =        21532     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.33220D+00    |proj g|=  1.49704D+00


 This problem is unconstrained.



At iterate   50    f=  7.05446D-01    |proj g|=  1.61916D-02

At iterate  100    f=  5.50259D-01    |proj g|=  2.34089D-02

At iterate  150    f=  4.73919D-01    |proj g|=  6.63617D-03

At iterate  200    f=  4.36572D-01    |proj g|=  3.26170D-03

At iterate  250    f=  4.15622D-01    |proj g|=  2.62995D-03

At iterate  300    f=  4.06518D-01    |proj g|=  8.17065D-03

At iterate  350    f=  4.01723D-01    |proj g|=  1.90510D-03

At iterate  400    f=  3.99333D-01    |proj g|=  2.77170D-03

At iterate  450    f=  3.97783D-01    |proj g|=  9.01702D-04

At iterate  500    f=  3.96904D-01    |proj g|=  4.49510D-04

At iterate  550    f=  3.96496D-01    |proj g|=  5.93727D-04

At iterate  600    f=  3.96233D-01    |proj g|=  5.38018D-04

At iterate  650    f=  3.96080D-01    |proj g|=  3.29728D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates

**MODEL 2 = BASELINE + DATA AUGMENTATION (28 classes + augmenté)**
---
split uniquement sur Y

In [11]:
# Assuming you have a dataset with features X and labels Y

# Split the dat using both Y and S using Y56_XL
X_train_, X_test_, Y_train_, Y_test_, S_train_, S_test_ = train_test_split(X, Y, S, test_size=0.2, random_state=42)

#refresh (if necessary)
X_train, X_test, Y_train, Y_test, S_train, S_test = X_train_, X_test_, Y_train_, Y_test_, S_train_, S_test_
Y56_train = Y_train + 28 * S_train

# Initialize SMOTE
sm = SMOTE(random_state=42)

# Fit the model to generate the data
X_train_XL, Y56_train_XL, = sm.fit_resample(X_train, Y56_train)
Y_train_XL = Y56_train_XL % 28  # reste (original Y)   ex 33% 28 = classe 5 
S_train_XL = Y56_train_XL//28   # facteur (original S) ex 33//28 = 1 (attribut protégé)

print(X_train_XL.shape,Y_train_XL.shape,S_train_XL.shape)

print(f'\ndistribution S (original sample): {len(S)}\n{np.bincount(S)} {np.round(np.bincount(S)/len(S)*100,2)}%')
print(f'\ndistribution S_XL (data augmentation): {len(S_train_XL)}\n{np.bincount(S_train_XL)} {np.round(np.bincount(S_train_XL)/len(S_train_XL)*100,2)}%')
Y56_grouped = np.bincount(Y56_train)
print(f'\ndistribution Y56 (56 classes): {len(Y56_train)}  (min {round(min(Y56_grouped)/len(Y56_grouped),1)}% , max {round(max(Y56_grouped)/len(Y56_grouped),1)}%)\n{Y56_grouped}')
print(f'\ndistribution Y56_XL (56 classes + data augmentation): {len(Y56_train_XL)} (x{round(len(Y56_train_XL)/len(Y56_train))})\n{np.bincount(Y56_train_XL)}')

# Normalize data (L2 norm recommended for embeddings)
#X = normalize(X, norm='l2')
#X_test_true = normalize(X_test_true, norm='l2')

(203728, 768) (203728,) (203728,)

distribution S (original sample): 27749
[14956 12793] [53.9 46.1]%

distribution S_XL (data augmentation): 203728
[101864 101864] [50. 50.]%

distribution Y56 (56 classes): 22199  (min 0.1% , max 65.0%)
[ 186  447 1096  101  122  267  539   16   75  282    6  573   69  104
  222   17  107   39  866 1172  208 3638  394   70  315  646  352   19
  120  129  719   47   36   47  277  192   14  126   65  543  345  954
  208  100   38   32  466 1135  198 3011  637    6   63  119  542   82]

distribution Y56_XL (56 classes + data augmentation): 203728 (x9)
[3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638
 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638
 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638
 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638]


In [12]:
# training logistic model 2
clf_2 = LogisticRegression(random_state=0, max_iter=5000,verbose=1).fit(X_train_XL, Y_train_XL)

print("Coefficients:", clf_2.coef_.shape)
print("Intercept:", clf_2.intercept_.shape)

# predicting and assessing
Y_pred_2 = clf_2.predict(X_test)
accuracy_2= accuracy_score(Y_test, Y_pred_2)  # Y_test are your original test labels
print(f"Accuracy on transformed test data: {accuracy_2}")
eval_scores_2, confusion_matrices_eval_2 = gap_eval_scores(Y_pred_2, Y_test, S_test, metrics=['TPR'])
final_score_2 = (eval_scores_2['macro_fscore']+ (1-eval_scores_2['TPR_GAP']))/2

#print results
print('final score',final_score_2)
print('macro_fscore',eval_scores_2['macro_fscore'])
print('1-eval_scores[\'TPR_GAP\']',1-eval_scores_2['TPR_GAP'])



RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =        21532     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.33220D+00    |proj g|=  1.93977D-02


 This problem is unconstrained.



At iterate   50    f=  3.88901D-01    |proj g|=  1.75555D-02

At iterate  100    f=  2.45014D-01    |proj g|=  1.28118D-02

At iterate  150    f=  1.83725D-01    |proj g|=  3.26611D-03

At iterate  200    f=  1.54198D-01    |proj g|=  2.75911D-03

At iterate  250    f=  1.39105D-01    |proj g|=  1.88299D-03

At iterate  300    f=  1.28366D-01    |proj g|=  3.43925D-03

At iterate  350    f=  1.21936D-01    |proj g|=  1.52751D-03

At iterate  400    f=  1.17522D-01    |proj g|=  8.83049D-04

At iterate  450    f=  1.14932D-01    |proj g|=  5.08136D-04

At iterate  500    f=  1.13282D-01    |proj g|=  9.50201D-04

At iterate  550    f=  1.12090D-01    |proj g|=  2.05493D-03

At iterate  600    f=  1.11323D-01    |proj g|=  4.32173D-04

At iterate  650    f=  1.10835D-01    |proj g|=  7.84113D-04

At iterate  700    f=  1.10492D-01    |proj g|=  1.11582D-03

At iterate  750    f=  1.10261D-01    |proj g|=  2.71370D-04

           * * *

Tit   = total number of iterations
Tnf   = total nu

**MODEL 3 = REGRESSION LOGISTIQUE (28 classes + YxS + augmenté)**
---

In [15]:
# Assuming you have a dataset with features X and labels Y

# Split the dat using both Y and S using Y56_XL
X_train_, X_test_, Y56_train_, Y56_test_, S_train_, S_test_ = train_test_split(X, Y56, S, test_size=0.2, random_state=42)
Y_train_ = Y56_train_ % 28  # reste (original Y)
Y_test_ = Y56_test_ % 28    # reste (original Y)

#refresh (if necessary)
X_train, X_test, Y_train, Y_test, S_train, S_test = X_train_, X_test_, Y_train_, Y_test_, S_train_, S_test_

# Initialize SMOTE
sm = SMOTE(random_state=42)

# Fit the model to generate the data
X_train_XL, Y56_train_XL = sm.fit_resample(X_train_, Y56_train_)
Y_train_XL = Y56_train_XL % 28  # reste (original Y)   ex 33% 28 = classe 5 
S_train_XL = Y56_train_XL//28               # facteur (original S) ex 33//28 = 1 (attribut protégé)

print(X_train_XL.shape,Y_train_XL.shape,S_train_XL.shape)

print(f'distribution S (original sample): {len(S)}\n{np.bincount(S)} {np.round(np.bincount(S)/len(S)*100,2)}%')
print(f'distribution S_XL (data augmentation): {len(S_train_XL)}\n{np.bincount(S_train_XL)} {np.round(np.bincount(S_train_XL)/len(S_train_XL)*100,2)}%')
Y56_grouped = np.bincount(Y56_train)
print(f'\ndistribution Y56 (56 classes): {len(Y56_train)}  (min {round(min(Y56_grouped)/len(Y56_grouped),1)}% , max {round(max(Y56_grouped)/len(Y56_grouped),1)}%)\n{Y56_grouped}')
print(f'\ndistribution Y56_XL (56 classes + data augmentation): {len(Y56_train_XL)} (x{round(len(Y56_train_XL)/len(Y56_train))})\n{np.bincount(Y56_train_XL)}')

# Normalize data (L2 norm recommended for embeddings)
#X = normalize(X, norm='l2')
#X_test_true = normalize(X_test_true, norm='l2')

(203728, 768) (203728,) (203728,)
distribution S (original sample): 27749
[14956 12793] [53.9 46.1]%
distribution S_XL (data augmentation): 203728
[101864 101864] [50. 50.]%

distribution Y56 (56 classes): 22199  (min 0.1% , max 65.0%)
[ 186  447 1096  101  122  267  539   16   75  282    6  573   69  104
  222   17  107   39  866 1172  208 3638  394   70  315  646  352   19
  120  129  719   47   36   47  277  192   14  126   65  543  345  954
  208  100   38   32  466 1135  198 3011  637    6   63  119  542   82]

distribution Y56_XL (56 classes + data augmentation): 203728 (x9)
[3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638
 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638
 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638
 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638]


In [16]:

# training logistic model
clf_3 = LogisticRegression(random_state=0, max_iter=5000,verbose=1).fit(X_train_XL, Y_train_XL)

print("Coefficients:", clf_3.coef_.shape)
print("Intercept:", clf_3.intercept_.shape)



RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =        21532     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.33220D+00    |proj g|=  1.93977D-02


 This problem is unconstrained.



At iterate   50    f=  3.88901D-01    |proj g|=  1.75555D-02

At iterate  100    f=  2.45014D-01    |proj g|=  1.28118D-02

At iterate  150    f=  1.83725D-01    |proj g|=  3.26611D-03

At iterate  200    f=  1.54198D-01    |proj g|=  2.75911D-03

At iterate  250    f=  1.39105D-01    |proj g|=  1.88299D-03

At iterate  300    f=  1.28366D-01    |proj g|=  3.43925D-03

At iterate  350    f=  1.21936D-01    |proj g|=  1.52751D-03

At iterate  400    f=  1.17522D-01    |proj g|=  8.83049D-04

At iterate  450    f=  1.14932D-01    |proj g|=  5.08136D-04

At iterate  500    f=  1.13282D-01    |proj g|=  9.50201D-04

At iterate  550    f=  1.12090D-01    |proj g|=  2.05493D-03

At iterate  600    f=  1.11323D-01    |proj g|=  4.32173D-04

At iterate  650    f=  1.10835D-01    |proj g|=  7.84113D-04

At iterate  700    f=  1.10492D-01    |proj g|=  1.11582D-03

At iterate  750    f=  1.10261D-01    |proj g|=  2.71370D-04

           * * *

Tit   = total number of iterations
Tnf   = total nu

In [17]:
# predicting and assessing
Y_pred_3 = clf_3.predict(X_test)
accuracy_3= accuracy_score(Y_test, Y_pred_3)  # Y_test are your original test labels
print(f"Accuracy on transformed test data: {accuracy_3}")
eval_scores_3, confusion_matrices_eval_3 = gap_eval_scores(Y_pred_3, Y_test, S_test, metrics=['TPR'])
final_score_3 = (eval_scores_3['macro_fscore']+ (1-eval_scores_3['TPR_GAP']))/2

#print results
print('final score',final_score_3)
print('macro_fscore',eval_scores_3['macro_fscore'])
print('1-eval_scores_3[\'TPR_GAP\']',1-eval_scores_3['TPR_GAP'])

Accuracy on transformed test data: 0.7282882882882883
gap_eval_scores
{'accuracy': 0.7282882882882883, 'macro_fscore': 0.643951515882604, 'micro_fscore': 0.7282882882882883}
distinct_groups size 2
{'TPR': array([0.64705877, 0.62365589, 0.79720278, 0.62962953, 0.55263155,
       0.80701744, 0.86725657, 0.66666611, 0.76470557, 0.79999991,
       0.25000062, 0.62499998, 0.41666674, 0.78260845, 0.59374997,
       0.25000125, 0.55999995, 0.56249992, 0.77327933, 0.71034481,
       0.59999996, 0.76221498, 0.70999996, 0.6923074 , 0.52631578,
       0.70440249, 0.46067417, 0.5       ]), 'TNR': array([0.99053094, 0.98284734, 0.98493754, 0.99496813, 0.99730639,
       0.99627245, 0.99447323, 0.99900066, 0.99832831, 0.99388379,
       0.99833333, 0.97555866, 0.99329758, 0.99329983, 0.99320652,
       0.99700399, 0.98927254, 0.99933155, 0.98623687, 0.97203826,
       0.99019608, 0.95160517, 0.98177441, 0.99966611, 0.98533424,
       0.97788697, 0.97499143, 0.99900199]), 'PPV': array([0.54098359, 0.

In [18]:
# (Re)Load the "true" test data
X_test_true = dat['X_test']
S_test_true = dat['S_test'] 

# Classify the provided test data with you classifier
y_test_true_3 = clf_3.predict(X_test_true)
results_3=pd.DataFrame(y_test_true_3, columns= ['score'])

results_3.to_csv("Data_Challenge_MDI_341_3.csv", header = None, index = None)
# np.savetxt('y_test_challenge_student.txt', y_test, delimiter=',')

**MODEL 4 = REGRESSION LOGISTIQUE (56 classes + YxS + augmenté)**
---

In [22]:
# Assuming you have a dataset with features X and labels Y

# Split the dat using both Y and S using Y56_XL
X_train_, X_test_, Y56_train_, Y56_test_, S_train_, S_test_ = train_test_split(X, Y56, S, test_size=0.2, random_state=42)
Y_train_ = Y56_train_ % 28  # reste (original Y)
Y_test_ = Y56_test_ % 28    # reste (original Y)

#refresh (if necessary)
X_train, X_test, Y_train, Y_test, S_train, S_test = X_train_, X_test_, Y_train_, Y_test_, S_train_, S_test_

# Initialize SMOTE
sm = SMOTE(random_state=42)

# Fit the model to generate the data
X_train_XL, Y56_train_XL = sm.fit_resample(X_train_, Y56_train_)
Y_train_XL = Y56_train_XL % 28  # reste (original Y)   ex 33% 28 = classe 5 
S_train_XL = Y56_train_XL//28               # facteur (original S) ex 33//28 = 1 (attribut protégé)

print(X_train_XL.shape,Y_train_XL.shape,S_train_XL.shape)

print(f'distribution S (original sample): {len(S)}\n{np.bincount(S)} {np.round(np.bincount(S)/len(S)*100,2)}%')
print(f'distribution S_XL (data augmentation): {len(S_train_XL)}\n{np.bincount(S_train_XL)} {np.round(np.bincount(S_train_XL)/len(S_train_XL)*100,2)}%')
Y56_grouped = np.bincount(Y56_train)
print(f'\ndistribution Y56 (56 classes): {len(Y56_train)}  (min {round(min(Y56_grouped)/len(Y56_grouped),1)}% , max {round(max(Y56_grouped)/len(Y56_grouped),1)}%)\n{Y56_grouped}')
print(f'\ndistribution Y56_XL (56 classes + data augmentation): {len(Y56_train_XL)} (x{round(len(Y56_train_XL)/len(Y56_train))})\n{np.bincount(Y56_train_XL)}')

# Normalize data (L2 norm recommended for embeddings)
#X = normalize(X, norm='l2')
#X_test_true = normalize(X_test_true, norm='l2')

(203728, 768) (203728,) (203728,)
distribution S (original sample): 27749
[14956 12793] [53.9 46.1]%
distribution S_XL (data augmentation): 203728
[101864 101864] [50. 50.]%

distribution Y56 (56 classes): 22199  (min 0.1% , max 65.0%)
[ 186  447 1096  101  122  267  539   16   75  282    6  573   69  104
  222   17  107   39  866 1172  208 3638  394   70  315  646  352   19
  120  129  719   47   36   47  277  192   14  126   65  543  345  954
  208  100   38   32  466 1135  198 3011  637    6   63  119  542   82]

distribution Y56_XL (56 classes + data augmentation): 203728 (x9)
[3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638
 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638
 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638
 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638 3638]


In [23]:

# training logistic model
clf_4 = LogisticRegression(random_state=0, max_iter=5000,verbose=1).fit(X_train_XL, Y56_train_XL)

print("Coefficients:", clf_3.coef_.shape)
print("Intercept:", clf_3.intercept_.shape)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =        43064     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.02535D+00    |proj g|=  1.11357D-02


 This problem is unconstrained.



At iterate   50    f=  3.42850D-01    |proj g|=  5.62567D-03

At iterate  100    f=  1.91289D-01    |proj g|=  3.03229D-03

At iterate  150    f=  1.39478D-01    |proj g|=  3.58310D-03

At iterate  200    f=  1.16121D-01    |proj g|=  1.61312D-03

At iterate  250    f=  1.03063D-01    |proj g|=  4.24770D-03

At iterate  300    f=  9.54912D-02    |proj g|=  1.50461D-03

At iterate  350    f=  9.03062D-02    |proj g|=  7.90046D-04

At iterate  400    f=  8.71011D-02    |proj g|=  4.92046D-04

At iterate  450    f=  8.48705D-02    |proj g|=  3.29116D-04

At iterate  500    f=  8.35554D-02    |proj g|=  5.41333D-04

At iterate  550    f=  8.26793D-02    |proj g|=  2.71808D-04

At iterate  600    f=  8.21284D-02    |proj g|=  2.82210D-04

At iterate  650    f=  8.17938D-02    |proj g|=  2.69958D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates

In [24]:
# predicting and assessing
Y56_pred_4 = clf_4.predict(X_test)

Y_pred_4 = Y56_pred_4 % 28  # reste (original Y)   ex 33% 28 = classe 5 

accuracy_4= accuracy_score(Y_test, Y_pred_4)  # Y_test are your original test labels
print(f"Accuracy on transformed test data: {accuracy_4}")
eval_scores_4, confusion_matrices_eval_4 = gap_eval_scores(Y_pred_4, Y_test, S_test, metrics=['TPR'])
final_score_4 = (eval_scores_4['macro_fscore']+ (1-eval_scores_4['TPR_GAP']))/2

#print results
print('final score',final_score_4)
print('macro_fscore',eval_scores_4['macro_fscore'])
print('1-eval_scores_4[\'TPR_GAP\']',1-eval_scores_4['TPR_GAP'])

Accuracy on transformed test data: 0.7317117117117117
gap_eval_scores
{'accuracy': 0.7317117117117117, 'macro_fscore': 0.6512454578298856, 'micro_fscore': 0.7317117117117117}
distinct_groups size 2
{'TPR': array([0.62745093, 0.65591394, 0.83566431, 0.62962953, 0.55263155,
       0.80701744, 0.87610613, 0.33333389, 0.58823519, 0.81538452,
       0.37500031, 0.63888887, 0.41666674, 0.73913023, 0.65624995,
       0.25000125, 0.40000008, 0.56249992, 0.77732791, 0.73103447,
       0.59999996, 0.78718783, 0.63999997, 0.84615331, 0.51315789,
       0.66666665, 0.31460678, 0.5       ]), 'TNR': array([0.9895164 , 0.98524871, 0.98420279, 0.99463267, 0.99764309,
       0.99762792, 0.99309153, 0.99933377, 0.99866265, 0.99320421,
       0.999     , 0.97486033, 0.99463807, 0.99497487, 0.99592391,
       0.99866844, 0.99430104, 1.        , 0.98587468, 0.96946284,
       0.98985801, 0.94633445, 0.97661623, 0.99899833, 0.98396998,
       0.98244998, 0.9756766 , 0.99966733]), 'PPV': array([0.50793651, 0

**MODEL 5 - REGRESSION LOGISTIQUE (56 classes + YxS)**
---
no data augmentation

In [25]:
# Assuming you have a dataset with features X and labels Y

# Split the dat using both Y and S using Y56_XL
X_train_, X_test_, Y56_train_, Y56_test_, S_train_, S_test_ = train_test_split(X, Y56, S, test_size=0.3, random_state=42)
Y_train_ = Y56_train_ % 28  # reste (original Y)
Y_test_ = Y56_test_ % 28    # reste (original Y)

#refresh (if necessary)
X_train, X_test, Y_train, Y_test, S_train, S_test = X_train_, X_test_, Y_train_, Y_test_, S_train_, S_test_

'''# Initialize SMOTE
sm = SMOTE(random_state=42)

# Fit the model to generate the data
X_train_XL, Y56_train_XL = sm.fit_resample(X_train_, Y56_train_)
Y_train_XL = Y56_train_XL % 28  # reste (original Y)   ex 33% 28 = classe 5 
S_train_XL = Y56_train_XL//28               # facteur (original S) ex 33//28 = 1 (attribut protégé)'''

print(X_train.shape,Y_train.shape,S_train.shape)

print(f'\ndistribution S (original sample): {len(S)}\n{np.bincount(S)} {np.round(np.bincount(S)/len(S)*100,2)}%')
print(f'\ndistribution Y56 (56 classes): {len(Y56)}  (min {round(min(Y56_grouped)/len(Y56_grouped),1)}% , max {round(max(Y56_grouped)/len(Y56_grouped),1)}%)\n{Y56_grouped}')


# Normalize data (L2 norm recommended for embeddings)
#X = normalize(X, norm='l2')
#X_test_true = normalize(X_test_true, norm='l2')

(19424, 768) (19424,) (19424,)

distribution S (original sample): 27749
[14956 12793] [53.9 46.1]%

distribution Y56 (56 classes): 27749  (min 0.1% , max 65.0%)
[ 186  447 1096  101  122  267  539   16   75  282    6  573   69  104
  222   17  107   39  866 1172  208 3638  394   70  315  646  352   19
  120  129  719   47   36   47  277  192   14  126   65  543  345  954
  208  100   38   32  466 1135  198 3011  637    6   63  119  542   82]


In [26]:
# predicting and assessing
clf_5 = LogisticRegression(random_state=0, max_iter=5000,verbose=1).fit(X_train, Y56_train_)

Y56_pred_5 = clf_5.predict(X_test)

Y_pred_5 = Y56_pred_5 % 28  # reste (original Y)   ex 33% 28 = classe 5 

accuracy_5= accuracy_score(Y_test, Y_pred_5)  # Y_test are your original test labels
print(f"Accuracy on transformed test data: {accuracy_5}")
eval_scores_5, confusion_matrices_eval_5 = gap_eval_scores(Y_pred_5, Y_test, S_test, metrics=['TPR'])
final_score_5 = (eval_scores_5['macro_fscore']+ (1-eval_scores_5['TPR_GAP']))/2

#print results
print('final score',final_score_5)
print('macro_fscore',eval_scores_5['macro_fscore'])
print('1-eval_scores_4[\'TPR_GAP\']',1-eval_scores_4['TPR_GAP'])

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =        43064     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.02535D+00    |proj g|=  8.32061D-01


 This problem is unconstrained.



At iterate   50    f=  7.21757D-01    |proj g|=  2.49188D-02

At iterate  100    f=  5.11279D-01    |proj g|=  1.04124D-02

At iterate  150    f=  4.21346D-01    |proj g|=  5.36256D-03

At iterate  200    f=  3.82613D-01    |proj g|=  6.31434D-03

At iterate  250    f=  3.65020D-01    |proj g|=  1.03230D-02

At iterate  300    f=  3.56433D-01    |proj g|=  1.67590D-03

At iterate  350    f=  3.52368D-01    |proj g|=  8.45603D-04

At iterate  400    f=  3.50371D-01    |proj g|=  2.84153D-03

At iterate  450    f=  3.49481D-01    |proj g|=  6.37972D-04

At iterate  500    f=  3.49067D-01    |proj g|=  6.02019D-04

At iterate  550    f=  3.48854D-01    |proj g|=  5.37719D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F   

**RESULTS**
---

In [47]:
RESULTS = pd.DataFrame(columns=['Model','Accuracy','final score','Fscore macro','1 - TPR gap'])
RESULTS.loc[0]=['baseline (Reglog28)',accuracy_0,final_score_0, eval_scores_0['macro_fscore'],1-eval_scores_0['TPR_GAP']]
RESULTS.loc[2]=['RegLog28 + YxS',accuracy_1,final_score_1, eval_scores_1['macro_fscore'],1-eval_scores_1['TPR_GAP']]
RESULTS.loc[4]=['RegLog28 + XL',accuracy_2,final_score_2, eval_scores_2['macro_fscore'],1-eval_scores_2['TPR_GAP']]
RESULTS.loc[5]=['RegLog28 + YxS + XL',accuracy_3,final_score_3, eval_scores_3['macro_fscore'],1-eval_scores_3['TPR_GAP']]
RESULTS.loc[6]=['RegLog56 + YxS + XL',accuracy_4,final_score_4, eval_scores_4['macro_fscore'],1-eval_scores_4['TPR_GAP']]
RESULTS.loc[3]=['RegLog56 + YxS',accuracy_5,final_score_5, eval_scores_5['macro_fscore'],1-eval_scores_5['TPR_GAP']]
RESULTS.loc[1]=['baseline seed=1 (Reglog28)',accuracy_0_1,final_score_0_1, eval_scores_0_1['macro_fscore'],1-eval_scores_0_1['TPR_GAP']]
RESULTS = RESULTS.sort_index()
RESULTS

,Model,Accuracy,final score,Fscore macro,1 - TPR gap
0,baseline (Reglog28),0.763363,0.732743,0.669528,0.795958
1,baseline seed=1 (Reglog28),0.857658,0.861761,0.840995,0.882527
2,RegLog28 + YxS,0.763363,0.732743,0.669528,0.795958
3,RegLog56 + YxS,0.751471,0.702264,0.653062,0.751467
4,RegLog28 + XL,0.728288,0.707930,0.643952,0.771908
5,RegLog28 + YxS + XL,0.728288,0.707930,0.643952,0.771908
6,RegLog56 + YxS + XL,0.731712,0.700916,0.651245,0.750586


In [28]:
with open('RESULTS_10-03-2024.pkl', 'wb') as f:
   pickle.dump(RESULTS, f)

#RESULTS =  pd.read_pickle('RESULTS_10-03-2024.pkl')

In [ ]:
# Keep the original index for later use
X['original_index'] = X.index

# Step 1: Keep original index by resetting it if not already in a suitable format
X.reset_index(drop=True, inplace=True)
Y = Y.reset_index(drop=True)
S = S.reset_index(drop=True)

# Create a combined Y and S DataFrame for stratification
YS = pd.DataFrame({'Y': Y, 'S': S})

# Split the data while stratifying based on Y and S
X_train, X_test, Y_train, Y_test, S_train, S_test = train_test_split(X, Y, S, test_size=0.3, random_state=0, stratify=YS)
print('train',X_train.shape,Y_train.shape,S_train.shape)
print('test',X_test.shape,Y_test.shape,S_test.shape)

# Create XS and XnotS subsets for training and test sets
XS_train = X_train[S == True]
YS_train = Y_train[S == True]

XnotS_train = X_train[S == False]
YnotS_train = Y_train[S == False]

XS_test = X_test[S == True]
YS_test = Y_test[S == True]

XnotS_test = X_test[S == False ]
XnotS_test = X_test[S == False ]

XS_train.head(5)

In [ ]:
print(Y_train[S == True].shape)
Y_train[S == True].head()

In [ ]:
# Prepare Logistic Regression models
model_X = LogisticRegression(random_state=0, max_iter=5000)
model_XS = LogisticRegression(random_state=0, max_iter=5000)
model_XnotS = LogisticRegression(random_state=0, max_iter=5000)

# Train the models
model_X.fit(X_train, Y_train)
model_XS.fit(XS_train, Y_train[S==1])
model_XnotS.fit(XnotS_train, Y_train[S==0])


In [ ]:
XF=X[S==1]
XH=X[S==0]
print(XF.shape,XH.shape)

YF=Y[S==1]
YH=Y[S==0]
print(YF.shape,YH.shape)

XF.head(5)

In [ ]:
# Evaluate model on test set
evaluation = model.evaluate(X_test, {'main_output': Y_test, 'adversary_output': S_test})


In [ ]:
# Extract shared representation model
representation_model = Model(inputs=model.input, outputs=model.get_layer('shared_representation').output)

# Transform X_train and X_test
X_train_transformed_2 = representation_model.predict(X_train)
X_test_transformed_2 = representation_model.predict(X_test)

In [ ]:
# Step 2: Train a new classifier on the transformed training data
clf_2 = LogisticRegression(max_iter=5000)  # Increase max_iter if needed for convergence
history_new_2 = clf_2.fit(X_train_transformed_2, Y_train)  # Y_train are your original training labels

# Step 3: Predict on the transformed test data and evaluate
Y_pred_2 = clf_2.predict(X_test_transformed_2)
accuracy_2= accuracy_score(Y_test, Y_pred_2)  # Y_test are your original test labels
print(f"Accuracy on transformed test data: {accuracy_2}")

# Step 4 : Predict with gloabl score
eval_scores_2, confusion_matrices_eval_2 = gap_eval_scores(Y_pred_2, Y_test, S_test, metrics=['TPR'])
final_score_2 = (eval_scores_2['macro_fscore']+ (1-eval_scores_2['TPR_GAP']))/2
print('\nfinal',final_score_2)
print('macro_fscore',eval_scores_2['macro_fscore'])
print('1-eval_scores',1-eval_scores_2['TPR_GAP'])

In [ ]:
classwise_loss(Y_test, Y_pred_2)
classwise_accuracy(Y_test, Y_pred_2)

In [ ]:
import tensorflow as tf

def classwise_accuracy(y_true, y_pred):
    """
    Custom metric function that calculates the accuracy for each class individually
    and then averages these accuracies.
    
    Args:
        y_true: Tensor of true labels, one-hot encoded.
        y_pred: Tensor of predicted labels, as probabilities.

    Returns:
        A scalar tensor representing the average class-wise accuracy.
    """
    # Convert probabilities to predicted class (highest probability)
    y_pred_classes = tf.argmax(y_pred, axis=-1)
    y_true_classes = tf.argmax(y_true, axis=-1)
    
    # Calculate accuracy for each class
    class_accuracies = tf.cast(tf.equal(y_true_classes, y_pred_classes), tf.float32)
    
    # Reduce across all dimensions but the first (batch dimension)
    # to get accuracy per class, then take the mean across classes
    classwise_acc = tf.reduce_mean(class_accuracies, axis=0)
    overall_acc = tf.reduce_mean(classwise_acc)
    
    return overall_acc

# Example usage with a model compilation in TensorFlow/Keras
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[classwise_accuracy])
